In [149]:
import time


In [150]:
import pandas as pd

start = time.time()

In [151]:
cpu_df = pd.read_csv('raw/cpu.csv')
motherboard_df = pd.read_csv('raw/mb.csv')
ram_df = pd.read_csv('raw/ram.csv')
gpu_df = pd.read_csv('raw/gpu.csv')
psu_df = pd.read_csv('raw/psu.csv')


FOR CPU

In [152]:
# Check unique values or first 20 rows
print(cpu_df['TDP'].unique()[:20])

# see which are non-numeric
print(cpu_df[~cpu_df['TDP'].apply(lambda x: str(x).replace('.','',1).isdigit())])

['65 W' '35 W' '105 W' '120 W' '170 W' '280 W' '60 W' '125 W' '100 W'
 '95 W' '220 W' '180 W' '250 W' '54 W' '46 W' '58 W' '47 W' '51 W' '62 W'
 '91 W']
                      Name        Price Producer               MPN  \
0        AMD Ryzen 5 5600X  $158.86 USD      AMD  100-100000065BOX   
1         AMD Athlon 3000G   $53.22 USD      AMD     YD3000C6FHBOX   
2        AMD Ryzen 3 3300X  $150.09 USD      AMD  100-100000159BOX   
3         AMD Ryzen 5 5500   $87.04 USD      AMD  100-100000457BOX   
4         AMD Ryzen 5 5600  $133.46 USD      AMD  100-100000927BOX   
..                     ...          ...      ...               ...   
323  Intel Xeon E5-2690 V4          NaN    Intel   BX80660E52690V4   
324  Intel Xeon E5-2695 V3          NaN    Intel   BX80644E52695V3   
325  Intel Xeon E5-2695 V4          NaN    Intel   BX80660E52695V4   
326  Intel Xeon E5-2697 V3          NaN    Intel   BX80644E52697V3   
327  Intel Xeon E5-2697 V4          NaN    Intel   BX80660E52697V4   

      

In [153]:
# Extract digits only
cpu_df['TDP'] = cpu_df['TDP'].astype(str).str.extract(r'(\d+)')[0]

# Convert to numeric
cpu_df['TDP'] = pd.to_numeric(cpu_df['TDP'], errors='coerce')

# Drop rows where conversion failed (NaN)
cpu_df = cpu_df.dropna(subset=['TDP'])

# Bin TDP
bins_cpu_tdp = [0, 65, 95, 125, 250]
labels_cpu_tdp = ['<=65W','66-95W','96-125W','>125W']
cpu_df['TDP_Bin'] = pd.cut(cpu_df['TDP'], bins=bins_cpu_tdp, labels=labels_cpu_tdp)

# Keep only needed columns
cpu_df = cpu_df[['Name','Producer','Socket','TDP_Bin']]

# Check the cleaned CPU dataframe
cpu_df.head()


,Name,Producer,Socket,TDP_Bin
0,AMD Ryzen 5 5600X,AMD,AM4,<=65W
1,AMD Athlon 3000G,AMD,AM4,<=65W
2,AMD Ryzen 3 3300X,AMD,AM4,<=65W
3,AMD Ryzen 5 5500,AMD,AM4,<=65W
4,AMD Ryzen 5 5600,AMD,AM4,<=65W


FOR GPU

In [154]:
# Check unique values or first 20 rows
print(gpu_df['TDP'].unique()[:20])

# see which are non-numeric
print(gpu_df[~gpu_df['TDP'].apply(lambda x: str(x).replace('.','',1).isdigit())])

['132 W' '220 W' '225 W' '165 W' '245 W' '263 W' '130 W' '170 W' '250 W'
 '200 W' '290 W' '320 W' '350 W' '480 W' '450 W' '115 W' '160 W' '650 W'
 '850 W' '107 W']
                                             Name        Price  Producer  \
0            Sapphire Pulse Radeon RX 6600 Gaming          NaN  SAPPHIRE   
1                           Aorus RTX 3070 Master          NaN  Gigabyte   
2                 ASRock Arc A750 Challenger D OC          NaN    ASRock   
3            ASRock Radeon RX 7600 Challenger 8GO          NaN    ASRock   
4        ASRock Radeon RX 7600 Phantom Gaming 8GO          NaN    ASRock   
...                                           ...          ...       ...   
1412       ZOTAC GeForce GTX 1080 Ti Arctic Storm          NaN     ZOTAC   
1413  ZOTAC GeForce GTX 1080 Ti Arctic Storm Mini          NaN     ZOTAC   
1414             ZOTAC GeForce GTX 1080 Ti Blower          NaN     ZOTAC   
1415   ZOTAC GeForce GTX 1080 Ti Founders Edition          NaN     ZOTAC   


In [155]:
gpu_df = gpu_df[['Name','Producer','Vram','TDP']]
# Drop rows with missing essential values
gpu_df.dropna(subset=['Vram','TDP'], inplace=True)

In [156]:
# Extract digits and convert to numeric
gpu_df['Vram'] = pd.to_numeric(gpu_df['Vram'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')
gpu_df['TDP'] = pd.to_numeric(gpu_df['TDP'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

# Drop rows that failed conversion
gpu_df = gpu_df.dropna(subset=['Vram','TDP'])


In [157]:
# Bin VRAM
bins_vram = [0, 4, 6, 8, 12, 24]
labels_vram = ['4GB','6GB','8GB','12GB','24GB+']
gpu_df['Vram_Bin'] = pd.cut(gpu_df['Vram'], bins=bins_vram, labels=labels_vram)

# Bin TDP
bins_gpu_tdp = [0, 150, 250, 350, 500]
labels_gpu_tdp = ['<=150W','151-250W','251-350W','>350W']
gpu_df['TDP_Bin'] = pd.cut(gpu_df['TDP'], bins=bins_gpu_tdp, labels=labels_gpu_tdp)

# Keep only the columns we need
gpu_df = gpu_df[['Name','Producer','Vram_Bin','TDP_Bin']]


In [158]:
# Check the cleaned GPU dataframe
gpu_df.head()

,Name,Producer,Vram_Bin,TDP_Bin
0,Sapphire Pulse Radeon RX 6600 Gaming,SAPPHIRE,8GB,<=150W
1,Aorus RTX 3070 Master,Gigabyte,8GB,151-250W
2,ASRock Arc A750 Challenger D OC,ASRock,8GB,151-250W
3,ASRock Radeon RX 7600 Challenger 8GO,ASRock,8GB,151-250W
4,ASRock Radeon RX 7600 Phantom Gaming 8GO,ASRock,8GB,151-250W


FOR MOTHERBOARD

In [159]:
# Check unique values or first 20 rows
print(motherboard_df['Chipset'].unique()[:20])

['B550' '970' 'A88X' 'B250' 'B360' 'B650' 'B650E' 'X399' 'H110' 'H97'
 'X299' 'X570' 'X99' 'Z270' 'Z370' 'Z390' 'Z790' 'Z97' 'B150' 'B365']


In [160]:
motherboard_df = motherboard_df[['Name','Producer','Socket','Chipset','Memory Type']]
# Drop rows with missing essential values
motherboard_df.dropna(subset=['Socket','Chipset','Memory Type'], inplace=True)

In [161]:
# Standardize Memory Type entries
motherboard_df['Memory Type'] = motherboard_df['Memory Type'].str.upper().str.extract(r'(DDR4|DDR5)')[0]


In [162]:
# Remove DDR3 and keep only DDR4 and DDR5 for modern compatibility
motherboard_df = motherboard_df[motherboard_df['Memory Type'].isin(['DDR4','DDR5'])]

In [163]:
motherboard_df.head()


,Name,Producer,Socket,Chipset,Memory Type
0,MSI B550-A Pro,MSI,AM4,B550,DDR4
3,ASRock B250 Pro4,ASRock,1151,B250,DDR4
4,ASRock B360 Pro4,ASRock,1151,B360,DDR4
5,ASRock B360M Pro4,ASRock,1151,B360,DDR4
6,ASRock B360M-ITX/AC,ASRock,1151,B360,DDR4


FOR RAM

In [164]:
# Check unique values or first 20 rows
print(ram_df['Ram Type'].unique()[:20])

['DDR4-3200' 'DDR5-5200' 'DDR5-4800' 'DDR4-3600' 'DDR4-2400' 'DDR3-1600'
 'DDR4-2666' 'DDR4-3466' 'DDR4-2800' 'DDR4-3000' 'DDR4-2133' 'DDR3-1866'
 'DDR4-3333' 'DDR4-3400' 'DDR4-5000' 'DDR4-4800' 'DDR1-333' 'DDR1-400'
 'DDR2-667' 'DDR2-533']


In [165]:
ram_df = ram_df[['Name','Producer','Ram Type','Size']]
# Drop rows with missing essential values
ram_df.dropna(subset=['Ram Type','Size'], inplace=True)

In [166]:
# Extract DDR type (DDR4 or DDR5)
ram_df['Ram Type'] = ram_df['Ram Type'].str.upper().str.extract(r'(DDR4|DDR5)')[0]

# Remove rows that are not DDR4 or DDR5
ram_df = ram_df[ram_df['Ram Type'].isin(['DDR4','DDR5'])]

In [167]:
# Extract digits from Size (in GB)
ram_df['Size'] = pd.to_numeric(ram_df['Size'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

# Drop rows where conversion failed
ram_df = ram_df.dropna(subset=['Size'])

# Bin RAM size
bins_size = [0, 4, 8, 16, 32, 64, 128]
labels_size = ['4GB','8GB','16GB','32GB','64GB','128GB+']
ram_df['Size_Bin'] = pd.cut(ram_df['Size'], bins=bins_size, labels=labels_size)

# Keep only necessary columns
ram_df = ram_df[['Name','Producer','Ram Type','Size_Bin']]


In [168]:
ram_df.head()


,Name,Producer,Ram Type,Size_Bin
0,Corsair Vengeance RGB RS,Corsair,DDR4,16GB
1,ADATA Lancer DDR5-5200,ADATA,DDR5,16GB
2,ADATA Premier DDR5-4800,ADATA,DDR5,16GB
3,ADATA XPG Spectrix D80 red,ADATA,DDR4,32GB
4,ADATA XPG Spectrix D80 black,ADATA,DDR4,16GB


FOR PSU

In [169]:
# Check unique values or first 20 rows
print(psu_df['Watt'].unique()[:20])

# see which are non-numeric
print(psu_df[~psu_df['Watt'].apply(lambda x: str(x).replace('.','',1).isdigit())])

['650 W' '750 W' '1200 W' '1000 W' '850 W' '300 W' '500 W' '400 W' '600 W'
 '700 W' '550 W' '450 W' '1600 W' '1300 W' '250 W' '350 W' '1050 W'
 '800 W' '90 W' '80 W']
                                            Name Price   Producer  \
0    Seasonic Core GM 80 Plus Gold, semi-modular   NaN   Seasonic   
1                             Aerocool Project 7   NaN   Aerocool   
2                             Aerocool Project 7   NaN   Aerocool   
3           be quiet! Dark Power Pro P11 modular   NaN  be quiet!   
4           be quiet! Dark Power Pro P11 modular   NaN  be quiet!   
..                                           ...   ...        ...   
932                                Zalman 850EBT   NaN     Zalman   
933             Zalman ZM1000-EBTII 80 PLUS Gold   NaN     Zalman   
934             Zalman ZM1200-EBTII 80 PLUS Gold   NaN     Zalman   
935              Zalman ZM700-EBTII 80 PLUS Gold   NaN     Zalman   
936              Zalman ZM800-EBTII 80 PLUS Gold   NaN     Zalman   

    

In [170]:
psu_df = psu_df[['Name','Producer','Watt']]
# Drop rows with missing Watt
psu_df.dropna(subset=['Watt'], inplace=True)

In [171]:
# Extract digits and convert to numeric
psu_df['Watt'] = pd.to_numeric(psu_df['Watt'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

# Drop rows that failed conversion
psu_df = psu_df.dropna(subset=['Watt'])


In [172]:
# Bin Watt
bins_psu = [0, 450, 600, 750, 900, 1200]
labels_psu = ['<=450W','451-600W','601-750W','751-900W','901-1200W']
psu_df['Watt_Bin'] = pd.cut(psu_df['Watt'], bins=bins_psu, labels=labels_psu)

# Keep only necessary columns
psu_df = psu_df[['Name','Producer','Watt_Bin']]

In [173]:
psu_df.head()

,Name,Producer,Watt_Bin
0,"Seasonic Core GM 80 Plus Gold, semi-modular",Seasonic,601-750W
1,Aerocool Project 7,Aerocool,601-750W
2,Aerocool Project 7,Aerocool,601-750W
3,be quiet! Dark Power Pro P11 modular,be quiet!,601-750W
4,be quiet! Dark Power Pro P11 modular,be quiet!,901-1200W


In [174]:
cpu_df.to_csv('cleaned/cpu_cleaned.csv', index=False)
gpu_df.to_csv('cleaned/gpu_cleaned.csv', index=False)
motherboard_df.to_csv('cleaned/mb_cleaned.csv', index=False)
ram_df.to_csv('cleaned/ram_cleaned.csv', index=False)
psu_df.to_csv('cleaned/psu_cleaned.csv', index=False)


Merge cleaned datasets into transactions

In [176]:
import random

transactions = []
target_size = 6000

cpu_df = cpu_df.reset_index(drop=True)
motherboard_df = motherboard_df.reset_index(drop=True)
ram_df = ram_df.reset_index(drop=True)
gpu_df = gpu_df.reset_index(drop=True)
psu_df = psu_df.reset_index(drop=True)

while len(transactions) < target_size:
    # CPU
    cpu = cpu_df.sample(n=1).iloc[0]
    
    # Compatible motherboard
    mb_compatible = motherboard_df[motherboard_df['Socket'] == cpu['Socket']]
    if mb_compatible.empty:
        continue
    mb = mb_compatible.sample(n=1).iloc[0]
    
    # Compatible RAM
    ram_compatible = ram_df[ram_df['Ram Type'] == mb['Memory Type']]
    if ram_compatible.empty:
        continue
    ram = ram_compatible.sample(n=1).iloc[0]
    
    # GPU & PSU
    gpu = gpu_df.sample(n=1).iloc[0]
    psu = psu_df.sample(n=1).iloc[0]
    
    # Combine attributes as items
    transaction = [
        f"CPU_Producer:{cpu['Producer']}", f"CPU_Socket:{cpu['Socket']}", f"CPU_TDP:{cpu['TDP_Bin']}",
        f"MB_Producer:{mb['Producer']}", f"MB_Socket:{mb['Socket']}", f"MB_Chipset:{mb['Chipset']}", f"MB_Memory:{mb['Memory Type']}",
        f"RAM_Producer:{ram['Producer']}", f"RAM_Type:{ram['Ram Type']}", f"RAM_Size:{ram['Size_Bin']}",
        f"GPU_Producer:{gpu['Producer']}", f"GPU_VRAM:{gpu['Vram_Bin']}", f"GPU_TDP:{gpu['TDP_Bin']}",
        f"PSU_Producer:{psu['Producer']}", f"PSU_Watt:{psu['Watt_Bin']}"
    ]
    transactions.append(transaction)

print("Generated transactions:", len(transactions))
print("Example transaction:", transactions[0])

Generated transactions: 6000
Example transaction: ['CPU_Producer:Intel', 'CPU_Socket:1151', 'CPU_TDP:66-95W', 'MB_Producer:ASUS', 'MB_Socket:1151', 'MB_Chipset:B360', 'MB_Memory:DDR4', 'RAM_Producer:Crucial', 'RAM_Type:DDR4', 'RAM_Size:32GB', 'GPU_Producer:ASUS', 'GPU_VRAM:6GB', 'GPU_TDP:151-250W', 'PSU_Producer:Kolink', 'PSU_Watt:nan']


In [177]:
import csv

with open('merged_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for t in transactions:
        writer.writerow(t)

One-hot encode transactions

In [178]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder


In [179]:
# Initialize encoder
te = TransactionEncoder()

# Fit and transform transactions
te_array = te.fit(transactions).transform(transactions)

# Convert to DataFrame
onehot_df = pd.DataFrame(te_array, columns=te.columns_)


In [180]:
print("Shape:", onehot_df.shape)
onehot_df.head()


Shape: (6000, 189)


,CPU_Producer:AMD,CPU_Producer:Intel,CPU_Socket:1151,CPU_Socket:1200,CPU_Socket:1700,CPU_Socket:2011-V3,CPU_Socket:2066,CPU_Socket:AM4,CPU_Socket:AM5,CPU_Socket:FM2+,...,RAM_Producer:Thermaltake,RAM_Size:128GB+,RAM_Size:16GB,RAM_Size:32GB,RAM_Size:4GB,RAM_Size:64GB,RAM_Size:8GB,RAM_Size:nan,RAM_Type:DDR4,RAM_Type:DDR5
0,False,True,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
1,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,True,False
3,False,True,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,False,True,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False


In [181]:
onehot_df = onehot_df.astype(int)

In [182]:
onehot_df.to_csv("processed_data.csv", index=False)

## Dataset Comparison: Before and After Preprocessing

This section compares the structure and quality of the dataset before preprocessing
(raw component CSV files) and after preprocessing (merged and one-hot encoded dataset).

In [183]:
import pandas as pd

cpu_raw = pd.read_csv("raw/cpu.csv")
gpu_raw = pd.read_csv("raw/gpu.csv")
mb_raw  = pd.read_csv("raw/mb.csv")
psu_raw = pd.read_csv("raw/psu.csv")
ram_raw = pd.read_csv("raw/ram.csv")

processed_df = pd.read_csv("processed_data.csv")

# BEFORE stats
before_stats = pd.DataFrame({
    "Rows": [
        cpu_raw.shape[0],
        gpu_raw.shape[0],
        mb_raw.shape[0],
        psu_raw.shape[0],
        ram_raw.shape[0]
    ],
    "Columns": [
        cpu_raw.shape[1],
        gpu_raw.shape[1],
        mb_raw.shape[1],
        psu_raw.shape[1],
        ram_raw.shape[1]
    ]
}, index=["CPU", "GPU", "Motherboard", "PSU", "RAM"])

# AFTER stats
after_stats = pd.DataFrame({
    "Rows (Transactions)": [processed_df.shape[0]],
    "Columns (Items)": [processed_df.shape[1]],
    "Sparsity": [
        round(1 - processed_df.sum().sum() / (processed_df.shape[0] * processed_df.shape[1]), 3)
    ]
}, index=["Processed Dataset"])

print("BEFORE PREPROCESSING")
display(before_stats)

print("\nAFTER PREPROCESSING")
display(after_stats)


BEFORE PREPROCESSING


,Rows,Columns
CPU,328,15
GPU,1417,19
Motherboard,1545,21
PSU,937,10
RAM,1674,12



AFTER PREPROCESSING


,Rows (Transactions),Columns (Items),Sparsity
Processed Dataset,6000,189,0.921


In [184]:
import pandas as pd

# ---------- Step 0: File Paths ----------
raw_files = ["raw/cpu.csv", "raw/gpu.csv", "raw/mb.csv", "raw/ram.csv", "raw/psu.csv"]
cleaned_files = ["cleaned/cpu_cleaned.csv", "cleaned/gpu_cleaned.csv", 
                 "cleaned/mb_cleaned.csv", "cleaned/ram_cleaned.csv", "cleaned/psu_cleaned.csv"]
merged_file = "merged_data.csv"
onehot_file = "processed_data.csv"

# ---------- Step 1: Raw Dataset Stats ----------
# Count rows per file (not actual transactions!)
raw_row_counts = [pd.read_csv(f).shape[0] for f in raw_files]
unique_counts = [pd.read_csv(f).shape[1] for f in raw_files]
print("Raw Dataset Stats:")
print("Raw component rows per file:", dict(zip(raw_files, raw_row_counts)))
print("Raw component unique attributes per file:", dict(zip(raw_files, unique_counts)))
# Note: Transactions not yet generated at this stage
print("Note: Transactions not yet generated at this stage.\n")

# ---------- Step 2: Cleaned Dataset Stats ----------
cleaned_row_counts = [pd.read_csv(f).shape[0] for f in cleaned_files]
print("Cleaned Dataset Stats:")
print("Cleaned component rows per file:", dict(zip(cleaned_files, cleaned_row_counts)))
print("Note: Transactions will be generated by merging compatible components.\n")

# ---------- Step 3: Merged Dataset Stats ----------
merged_df = pd.read_csv(merged_file)
merged_transactions = len(merged_df)
merged_columns = merged_df.shape[1]
print("Merged Dataset Stats:")
print(f"Transactions (full PC builds): {merged_transactions}")
print(f"Attribute columns: {merged_columns}")
print("Unique items (columns = attributes):", merged_columns, "\n")

# ---------- Step 4: One-Hot Encoded Dataset Stats ----------
onehot_df = pd.read_csv(onehot_file)
onehot_transactions = onehot_df.shape[0]
onehot_items = onehot_df.shape[1]
density = onehot_df.values.sum() / (onehot_transactions * onehot_items)
print("One-Hot Encoded Dataset Stats:")
print(f"Transactions: {onehot_transactions}")
print(f"Unique Items (one-hot columns): {onehot_items}")
print(f"Density: {density:.4f}")

# ---------- Optional: Before/After Table ----------
stats_table = pd.DataFrame({
    "Dataset": ["Merged", "One-Hot Encoded"],
    "Transactions": [merged_transactions, onehot_transactions],
    "Unique Items": [merged_columns, onehot_items],
    "Density": ["-", f"{density:.4f}"]
})
print("\nComparison Table:")
print(stats_table)


Raw Dataset Stats:
Raw component rows per file: {'raw/cpu.csv': 328, 'raw/gpu.csv': 1417, 'raw/mb.csv': 1545, 'raw/ram.csv': 1674, 'raw/psu.csv': 937}
Raw component unique attributes per file: {'raw/cpu.csv': 15, 'raw/gpu.csv': 19, 'raw/mb.csv': 21, 'raw/ram.csv': 12, 'raw/psu.csv': 10}
Note: Transactions not yet generated at this stage.

Cleaned Dataset Stats:
Cleaned component rows per file: {'cleaned/cpu_cleaned.csv': 328, 'cleaned/gpu_cleaned.csv': 1417, 'cleaned/mb_cleaned.csv': 1399, 'cleaned/ram_cleaned.csv': 1390, 'cleaned/psu_cleaned.csv': 937}
Note: Transactions will be generated by merging compatible components.

Merged Dataset Stats:
Transactions (full PC builds): 5999
Attribute columns: 15
Unique items (columns = attributes): 15 

One-Hot Encoded Dataset Stats:
Transactions: 6000
Unique Items (one-hot columns): 189
Density: 0.0794

Comparison Table:
           Dataset  Transactions  Unique Items Density
0           Merged          5999            15       -
1  One-Hot Enco

| Aspect           | Raw Data        | After Preprocessing |
| ---------------- | --------------- | ------------------- |
| Number of files  | 5 separate CSVs | 1 unified dataset   |
| Data granularity | Component-level | PC build-level      |
| Compatibility    | Not enforced    | Enforced            |
| Numeric values   | Continuous      | Discretized         |
| Format           | Relational      | Transactional       |
| Apriori-ready    | ❌              | ✅                 |


In [185]:
preprocessing_time = time.time() - start
print(f"Preprocessing Time: {preprocessing_time:.2f} seconds")

Preprocessing Time: 21.05 seconds
